In [1]:
import numpy as np
import pandas as pd

In [ ]:
class DeltaQuery:
    query_types = set(['CREATE', 'UPDATE', 'DELETE'])
    query_subtypes = {
        'CREATE': set(['PARTY', 'ACCOUNT', 
                    'PERSONAL_INFO', 'PARTY_ADDRESS', 
                    'ACCOUNT_ADDRESS', 'EMPLOYMENT_ADDRESS',
                    'EMPLOYMENT', 'PRODUCT',
                    'REFERRAL', 'INVESTMENT_EXPERIENCE',
                    'EMPLOYMEE_RELATIONSHIP', 'INTERESTED_PARTY',
                    'BENEFICIARY', 'TRUSTED_CONTACT', 
                    'DIRECT_TRUSTED_CONTACT']),
        'UPDATE': set(['PARTY', 'ACCOUNT',
                    'PERSONAL_INFO', 'PARTY_ADDRESS',
                    'PARTY_ADDRESS', 'EMPLOYMENT_ADDRESS',
                    'EMPLOYMENT', 'PRODUCT',
                    'INVESTMENT_EXPERIENCE', 'BENEIFICIARY',
                    'TRUSTED_CONTACT', 'DIRECT_TRUSTED_CONTACT']),
        'DELETE': set(['PARTY', 'ACCOUNT',
                    'PERSONAL_INFO', 'PARTY_ADDRESS',
                    'PARTY_ADDRESS', 'EMPLOYMENT_ADDRESS',
                    'EMPLOYMENT', 'PRODUCT',
                    'REFERRAL', 'INVESTMENT_EXPERIENCE',
                    'EMPLOYMEE_RELATIONSHIP', 'INTERESTED_PARTY',
                    'BENEFICIARY', 'TRUSTED_CONTACT',
                    'DIRECT_TRUSTED_CONTACT'])
    }
    
    required_keys = {
        'PARTY': ['partyId', ],
        'ACCOUNT': ['accountId', 'partyId'],
        'PERSONAL_INFO': ['partyId', ],
        'PARTY_ADDRESS': ['partyId', ],
        'ACCOUNT_ADDRESS': ['accountId', ],
        'EMPLOYMENT_ADDRESS': ['partyId', ],
        'EMPLOYMENT': ['partyId', ],
        'PRODUCT': ['accountId', ],
        'REFERRAL': ['partyId', ],
        'INVESTMENT_EXPERIENCE': ['partyId', ],
        'EMPLOYMEE_RELATIONSHIP': ['partyId', ],
        'INTERESTED_PARTY': ['partyId', 'interestedPartyId', ],
        'BENEFICIARY': ['accountId', 'beneficiaryId', ],
        'TRUSTED_CONTACT': ['accountId', 'trustedContactPartyId', ],
        'DIRECT_TRUSTED_CONTACT': ['partyId', 'directTrustedContactPartyId', ],
    }

    node_maps = {
        'PARTY': "Party",
        'ACCOUNT': "Account",
        'PERSONAL_INFO': "PersonalInfo",
        'PARTY_ADDRESS': "Address",
        'ACCOUNT_ADDRESS': "Address",
        'EMPLOYMENT_ADDRESS': "Address",
        'EMPLOYMENT': "Employment",
        'PRODUCT': "Product",
        'REFERRAL': "Referral",
        'INVESTMENT_EXPERIENCE': "InvestmentExperience",
        'EMPLOYMEE_RELATIONSHIP': "EmployeeRelationship",
    }

    edge_maps_query_template = {
        'INTERESTED_PARTY': """
            MATCH (a:Account {id: {accountId}}), 
                (p:Party {id: {partyId}}) 
            CREATE (a)-[:INTERESTED_PARTY {data}]->(p)""",
        'BENEFICIARY': """
            MATCH (a:Account {id: {accountId}}), 
                (p:Party {id: {partyId}}) 
            CREATE (a)-[:HAS_BENEFICIARY {data}]->(p)""",
        'TRUSTED_CONTACT': """
            MATCH (a:Account {id: {accountId}}), 
                (p:Party {id: {partyId}}) 
            CREATE (a)-[:TRUSTED_CONTACT {data}]->(p)""",
        'DIRECT_TRUSTED_CONTACT': """
            MATCH (p:Party {id: {partyId}}), 
                (p2:Party {id: {directTrustedContactPartyId}}) 
            CREATE (p)-[:DIRECT_TRUSTED_CONTACT {data}]->(p2)"""
    }



    
    def __init__(self, type, subtype, keys, data):
        self.type = type
        self.subtype = subtype
        assert self.type_check(), f'Invalid query type: {self.type}, subtype: {self.subtype}'

        self.keys = keys
        assert self.key_check(), f'Invalid keys for {self.subtype} query'

        self.data = data

    
    def type_check(self):
        return self.type in DeltaQuery.query_types and self.subtype in DeltaQuery.query_subtypes[self.type]
    
    
    def key_check(self):
        return (len(self.keys) == DeltaQuery.required_keys[self.subtype] and 
                all([k in self.keys for k in DeltaQuery.required_keys[self.subtype]]))
    

    def __to_cypher_query(self):
        if self.type == 'CREATE':
            return self.__to_cypher_query_create()
        elif self.type == 'UPDATE':
            return self.__to_cypher_query_update()
        elif self.type == 'DELETE':
            return self.__to_cypher_query_delete()
        else:
            return None
    
    def __to_cypher_query_create(self) :
        if self.subtype in DeltaQuery.node_maps :
            query = """
                CREATE (n:{subtype} {data}) 
                """
            query = query.format(
                subtype=DeltaQuery.node_maps[self.subtype], 
                data=self.data)
            
            #TODO: Fix Attribute Names
            if self.subtype == 'ACCOUNT' :
                query += """
                    MATCH (p:Party {id: {partyId}})
                    CREATE (p)-[:HAS_ACCOUNT]->(n)
                    CREATE (n)-[:OWNED_BY]->(p)
                    """.format(partyId=self.keys['PartyId'])
            elif self.subtype == 'PERSONAL_INFO' :
                query += """
                    MATCH (p:Party {id: {partyId}})
                    CREATE (p)-[:DETAILS]->(n)
                    """.format(partyId=self.keys['PartyId'])
            elif self.subtype == 'PARTY_ADDRESS' :
                query += """
                    MATCH (p:Party {id: {partyId}})
                    CREATE (p)-[:PARTY_ADDRESS]->(n)
                    """.format(partyId=self.keys['PartyId'])
            elif self.subtype == 'ACCOUNT_ADDRESS' :
                query += """
                    MATCH (a:Account {id: {accountId}})
                    CREATE (a)-[:ACCOUNT_ADDRESS]->(n)
                    """.format(accountId=self.keys['accountId'])
            elif self.subtype == 'EMPLOYMENT': 
                query += """
                    MATCH (p:Party {id: {partyId}})-[:DETAILS]->(pi)
                    CREATE (pi)-[:HAS_EMPLOYMENT]->(n)
                    """.format(partyId=self.keys['partyId'])
            elif self.subtype == 'PRODUCT':
                query += """
                    MATCH (a:Account {id: {accountId}})
                    CREATE (a)-[:HAS_PRODUCT]->(n)
                    """.format(accountId=self.keys['accountId'])
            elif self.subtype == 'REFERRAL':
                query += """
                    MATCH (p:Account {id: {accountId}})
                    CREATE (p)-[:REFERRED_BY]->(n)
                    """.format(partyId=self.keys['partyId'])
            elif self.subtype == 'INVESTMENT_EXPERIENCE':
                query += """
                    MATCH (p:Party {id: {partyId}})
                    CREATE (p)-[:INVESTMENT_EXPERIENCE]->(n)
                    """.format(partyId=self.keys['partyId'])
            elif self.subtype == 'EMPLOYMEE_RELATIONSHIP':
                query += """
                    MATCH (p:Party {id: {partyId}})
                    CREATE (p)-[:EMPLOYEE]->(n)
                    """.format(partyId=self.keys['partyId'])
            return query
        elif self.subtype in DeltaQuery.edge_maps :
            self.keys['accountId'] = self.keys.get('accountId', None)
            self.keys['partyId'] = self.keys.get('partyId', None)
            self.keys['directTrustedContactPartyId'] = self.keys.get('directTrustedContactPartyId', None)
            query = DeltaQuery.edge_maps[self.subtype].format(
                data=self.data,
                accountId=self.keys['accountId'],
                directTrustedContactPartyId=self.keys['directTrustedContactPartyId'],
                partyId=self.keys['partyId']
            )
            return query
            

    def __to_cypher_query_update() :
        base_match_query = DeltaQuery.match_queries[self.subtype]

    def __to_cypher_query_delete() :
        pass

In [ ]:
class DeltaLoader:
    def __init__(self, query_queue):
        self.query_queue = query_queue
    
    def add_to_queue(self, query):
        self.query_queue.append(query)
    
    